In [1]:
gpu_info = !nvidia-smi
gpu_info = gpu_info[:10]
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Sun Nov  1 23:00:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import torch
import os
import numpy as np
import torch.nn as nn
import torch.optim as optim

from tqdm.notebook import tqdm_notebook as tq

from torch.autograd import Variable
from torchvision.transforms import transforms
from torchvision.utils import save_image, make_grid
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary

In [4]:
base_path = "/content/drive/My Drive/UC Davis Synthetic Data/Prashanth's/Autoencoder/Autoencoder/"
sys.path.append(base_path)
# torch.manual_seed(123)

In [5]:
from src.data_utils import LoadDataset, SavePath
from src.config import TrainConfig

In [6]:
args = TrainConfig( base_path,              # project directory path
                    n_epochs = 200,         # number of epochs to train (default: 100)
                    batch_size = 128,       # input batch size for training (default: 128)
                    lr = 0.0002,            # learning rate (default: 0.0001)
                    dim_h = 128,            # hidden dimension (default: 128)')
                    n_z = 8,                # hidden dimension of z (default: 8)
                    LAMBDA = 10,            # regularization coef term (default: 10)
                    sigma = 1,              # variance of hidden dimension (default: 1)
                    n_channel = 1,          # input channels (default: 1)
                    img_size = 28 )         # image size

In [7]:
def unfreeze_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = True

def freeze_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = False

In [8]:
def save_models(model_path, epoch_no):
    print("Saving models")
    torch.save(conv_encoder.state_dict(), model_path + "/conv_encoder_" + "%d.pth" % epoch_no)
    torch.save(conv_decoder.state_dict(), model_path + "/conv_decoder_" + "%d.pth" % epoch_no)
    torch.save(linear_encoder.state_dict(), model_path + "/linear_encoder_" + "%d.pth" % epoch_no)
    torch.save(linear_decoder.state_dict(), model_path + "/linear_decoder_" + "%d.pth" % epoch_no)
    torch.save(discriminator.state_dict(), model_path + "/discriminator_" + "%d.pth" % epoch_no)

def save_lists(list_path, epoch_no, reconstr_loss):
    print("Saving list")
    if reconstr_loss:
        np.savetxt(list_path + '/reconstr_loss_' + '%d.txt'% epoch_no, reconstr_loss)

In [9]:
sp = SavePath(args, checkpoint_path="/content/drive/My Drive/UC Davis Synthetic Data/Prashanth's/Autoencoder/Autoencoder/outs/Sun-Oct-18-16-57-26-2020/")
# sp = SavePath(args)

/content/drive/My Drive/UC Davis Synthetic Data/Prashanth's/Autoencoder/Autoencoder/outs/Sun-Oct-18-16-57-26-2020/


In [10]:
# The output of torchvision datasets are PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))])

cdl = LoadDataset(args, data='mnist', transform=None)
train_loader = cdl.get_data_loader(train=True)
test_loader = cdl.get_data_loader(train=False)

# cdl_norm = LoadDataset(args, data='mnist', transform=transform)
# train_loader_norm = cdl_norm.get_data_loader(train=True)
# test_loader_norm = cdl_norm.get_data_loader(train=False)

In [11]:
class LinearEncoder(nn.Module):

    def __init__(self, args):
        super(LinearEncoder, self).__init__()
        
        self.n_z = args.n_z
        self.dim_h = args.dim_h
        self.dim_input = args.img_size ** 2

        self.main = nn.Sequential(
            nn.Linear(self.dim_input * 2, self.dim_h * 16),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 16, self.dim_h * 8),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 8, self.dim_h * 8),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 8, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 2),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 2, self.dim_h),
            nn.ReLU(True),
            nn.Linear(self.dim_h, self.n_z)
        )

        # self.preprocess = nn.Sequential(
        #     nn.Linear(self.dim_input * 2, self.dim_input),
        #     nn.ReLU(True)
        # )
    
    def forward(self, x):
        # x = self.preprocess(x)
        x = self.main(x)
        return x

In [12]:
class LinearDecoder(nn.Module):

    def __init__(self, args):
        super(LinearDecoder, self).__init__()

        self.n_z = args.n_z
        self.dim_h = args.dim_h
        self.dim_output = args.img_size ** 2

        self.main = nn.Sequential(
            nn.Linear(self.n_z, self.dim_h),
            nn.ReLU(True),
            nn.Linear(self.dim_h, self.dim_h * 2),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 2, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 8),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 8, self.dim_h * 8),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 8, self.dim_h * 16),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 16, self.dim_output * 2),
            # nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.main(x)
        return x

In [13]:
class Discriminator(nn.Module):
    def __init__(self, args):
        super(Discriminator, self).__init__()

        self.dim_h = args.dim_h
        self.n_z = args.n_z

        self.main = nn.Sequential(
            nn.Linear(self.n_z, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, self.dim_h * 4),
            nn.ReLU(True),
            nn.Linear(self.dim_h * 4, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.main(x)
        return x

In [14]:
from src.model import Encoder as ConvEncoder
from src.model import Decoder as ConvDecoder

In [20]:
conv_encoder, conv_decoder = ConvEncoder(args).cuda(), ConvDecoder(args).cuda()
linear_encoder, linear_decoder = LinearEncoder(args).cuda(), LinearDecoder(args).cuda()
discriminator = Discriminator(args).cuda()

criterion = nn.MSELoss()

models = [conv_encoder, conv_decoder, linear_encoder, linear_decoder, discriminator]
for i in range(len(models)):
    models[i].train()

# Optimizers
optimizers = [optim.Adam(models[i].parameters(), lr = args.lr) for i in range(len(models)-1)]
optimizers.append(optim.Adam(discriminator.parameters(), lr = 0.5*args.lr))

In [ ]:
# summary(discriminator, (1, 8))
# summary(linear_decoder, (1, 8))
# summary(linear_encoder, (1, 784*2))

In [21]:
checkpoint = 0
reconstr_loss_epoch = []
if checkpoint:
    _, list_path, model_path = sp.get_save_paths()
    linear_encoder.load_state_dict(torch.load(model_path + "/linear_encoder_{}.pth".format(checkpoint)))
    conv_encoder.load_state_dict(torch.load(model_path + "/conv_encoder_{}.pth".format(checkpoint)))
    linear_decoder.load_state_dict(torch.load(model_path + "/linear_decoder_{}.pth".format(checkpoint)))
    conv_decoder.load_state_dict(torch.load(model_path + "/conv_decoder_{}.pth".format(checkpoint)))
    discriminator.load_state_dict(torch.load(model_path + "/discriminator_{}.pth".format(checkpoint)))
    # reconstr_loss_epoch = np.loadtxt(list_path + '/reconstr_loss_{}.txt'.format(checkpoint)).tolist()
reconstr_loss = []
disc_loss = []
d_real_val = []

In [22]:
image_path, list_path, model_path = sp.get_save_paths()

In [23]:
for epoch in range(checkpoint, checkpoint+args.n_epochs):
    for step, (images, _) in tq(enumerate(train_loader)):

        images = images.cuda()
        images_flattened = images.reshape([images.size()[0], 1, args.img_size**2, 1])
        images_complex = torch.cat((images_flattened, torch.zeros_like(images_flattened)), 3)
        images_fft = images_complex.fft(signal_ndim=1, normalized=True).reshape([images_complex.size()[0], 1, (args.img_size**2)*2])
        
        for i in range(len(models)):
            models[i].zero_grad()

        # ======== Train Discriminator ======== #

        unfreeze_params(discriminator)
        for i in range(len(models)-1):
            freeze_params(models[i])

        z_dist = torch.randn(images.size()[0], args.n_z) * args.sigma
        z_dist = z_dist.cuda().detach()

        z_linear = linear_encoder(images_fft).detach()
        z_conv = conv_encoder(images).detach()

        d_dist = discriminator(z_dist)
        d_linear = discriminator(z_linear)
        d_conv = discriminator(z_conv)
        
        d_loss = -(torch.log(d_dist.mean()) + torch.log((1 - d_linear).mean()) + torch.log((1 - d_conv).mean()))

        disc_loss.append([d_dist.mean().item(), d_linear.mean().item(), d_conv.mean().item()])

        d_loss.backward()

        optimizers[4].step()

        # ======== Train Generator ======== #
        
        freeze_params(discriminator)
        for i in range(len(models)-1):
            unfreeze_params(models[i])

        batch_size = images.size()[0]

        z_conv_real = conv_encoder(images)
        z_linear_real = linear_encoder(images_fft)

        recon_conv = conv_decoder(z_conv_real)
        recon_linear = linear_decoder(z_linear_real)
        
        d_conv_real = discriminator(z_conv_real)
        d_linear_real = discriminator(z_linear_real)

        recon_loss_conv = criterion(recon_conv, images)
        recon_loss_linear = criterion(recon_linear, images_fft)
        recon_loss = 5 * recon_loss_conv + recon_loss_linear
        # - args.LAMBDA * (torch.log(d_real)).mean()

        reg_loss = -(args.LAMBDA * (torch.log(d_conv_real).mean() + torch.log(d_linear_real).mean()))

        distance_loss = args.LAMBDA * criterion(z_conv_real, z_linear_real.squeeze())

        total_loss = recon_loss + reg_loss + distance_loss
        # recon_loss.backward()
        # reg_loss.backward()
        # distance_loss.backward()

        total_loss.backward()

        d_real_val.append([reg_loss.item(), distance_loss.item()])
        reconstr_loss.append([recon_loss_conv.data.item(), recon_loss_linear.data.item()])

        for i in range(len(optimizers)-1):
            optimizers[i].step()
            

    if (epoch + 1) % 1 == 0:
        temp0 = np.mean(reconstr_loss, axis=0)
        temp = np.mean(disc_loss, axis=0)
        temp1 = np.mean(d_real_val, axis=0)
        print("Epoch: [%d/%d], Recon Loss: [%.4f, %.4f], reg_loss: [%.4f], distance_loss: [%.4f], d_dist: [%.4f], d_linear: [%.4f], d_conv: [%.4f]" %
                (epoch + 1, args.n_epochs, 
                 temp0[0], temp0[1], temp1[0], temp1[1], temp[0], temp[1], temp[2]))
        disc_loss.clear()
        d_real_val.clear()
    
    if (epoch + 1) % 1 == 0:

        # reconstr_loss_epoch.append(np.mean(reconstr_loss))
        # reconstr_loss.clear()

        batch_size = args.batch_size // 2
        test_iter = iter(test_loader)
        test_images, _ = next(test_iter)
        test_images = Variable(test_images)[:64]
        test_images_flattened = test_images[:64].reshape([batch_size, 1, args.img_size**2, 1])

        test_images_complex = torch.cat((test_images_flattened, torch.zeros_like(test_images_flattened)), 3)
        test_images_fft = test_images_complex.fft(signal_ndim=1, normalized=True).reshape([batch_size, 1, (args.img_size**2)*2])

        z_conv = conv_encoder(test_images.cuda()).detach()
        z_linear = linear_encoder(test_images_fft.cuda()).detach()

        z_dist = torch.randn_like(z_linear).cuda()

        recon_linear = linear_decoder(z_linear).reshape([batch_size, 1, args.img_size**2, 2]).ifft(signal_ndim=1, normalized=True).sum(3)
        recon_conv_linear = linear_decoder(z_conv).reshape([batch_size, 1, args.img_size**2, 2]).ifft(signal_ndim=1, normalized=True).sum(3)

        z_linear_sample = linear_decoder(z_dist).reshape([batch_size, 1, args.img_size**2, 2]).ifft(signal_ndim=1, normalized=True).sum(3)

        conv_sample = make_grid(conv_decoder(z_dist).detach().cpu().view(batch_size, 1, 28, 28))
        linear_sample = make_grid(z_linear_sample.detach().cpu().view(batch_size, 1, 28, 28))
        sample = torch.cat((conv_sample, linear_sample), axis=1)
 
        image = torch.cat((test_images.view(batch_size, 1, 28, 28), 
                           conv_decoder(z_conv).cpu().view(batch_size, 1, 28, 28).data,
                           recon_linear.cpu().view(batch_size, 1, 28, 28).data,
                           conv_decoder(z_linear).cpu().view(batch_size, 1, 28, 28).data,
                           recon_conv_linear.cpu().view(batch_size, 1, 28, 28).data
                           ), axis=3)
        
        save_image(image, image_path +
                                '/inputs_reconstr_{}.png'.format(epoch+1), normalize=False, pad_value=1)
        
        save_image(sample, image_path + 
                                '/sample_{}.png'.format(epoch+1), normalize=False, pad_value=1)

    if (epoch + 1) % 25 == 0:
        save_models(model_path, epoch+1)
        # save_lists(list_path, epoch+1, reconstr_loss_epoch)
    


Epoch: [1/200], Recon Loss: [0.0747, 0.0345], reg_loss: [38.0077], distance_loss: [8.3563], d_dist: [0.2682], d_linear: [0.1868], d_conv: [0.4053]



Epoch: [2/200], Recon Loss: [0.0692, 0.0337], reg_loss: [30.0957], distance_loss: [5.6583], d_dist: [0.2811], d_linear: [0.3443], d_conv: [0.3161]



Epoch: [3/200], Recon Loss: [0.0668, 0.0332], reg_loss: [25.4714], distance_loss: [5.0402], d_dist: [0.3585], d_linear: [0.4288], d_conv: [0.2781]



Epoch: [4/200], Recon Loss: [0.0652, 0.0331], reg_loss: [30.0552], distance_loss: [5.9244], d_dist: [0.3042], d_linear: [0.4176], d_conv: [0.2436]



Epoch: [5/200], Recon Loss: [0.0636, 0.0328], reg_loss: [24.1225], distance_loss: [4.3276], d_dist: [0.3566], d_linear: [0.4386], d_conv: [0.2726]


...


Epoch: [196/200], Recon Loss: [0.0331, 0.0171], reg_loss: [22.1817], distance_loss: [0.3536], d_dist: [0.3312], d_linear: [0.3379], d_conv: [0.3260]



Epoch: [197/200], Recon Loss: [0.0330, 0.0171], reg_loss: [22.0390], distance_loss: [0.2848], d_dist: [0.3329], d_linear: [0.3379], d_conv: [0.3294]



Epoch: [198/200], Recon Loss: [0.0330, 0.0170], reg_loss: [22.1082], distance_loss: [0.2814], d_dist: [0.3331], d_linear: [0.3365], d_conv: [0.3287]



Epoch: [199/200], Recon Loss: [0.0329, 0.0170], reg_loss: [22.0154], distance_loss: [0.3242], d_dist: [0.3341], d_linear: [0.3396], d_conv: [0.3292]



Epoch: [200/200], Recon Loss: [0.0329, 0.0170], reg_loss: [22.1601], distance_loss: [0.2760], d_dist: [0.3313], d_linear: [0.3354], d_conv: [0.3279]
Saving models
